<a href="https://colab.research.google.com/github/fowardelcac/Abandono-Escolar/blob/main/Notebooks/Modelado_b_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/fowardelcac/Abandono-Escolar.git

Cloning into 'Abandono-Escolar'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 82 (delta 25), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (82/82), 1.04 MiB | 3.34 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

In [3]:
dataset_original = pd.read_csv('https://raw.githubusercontent.com/fowardelcac/Abandono-Escolar/main/Dataset/dataset.csv')
df_editado = pd.read_csv('https://raw.githubusercontent.com/fowardelcac/Abandono-Escolar/main/Dataset/D2/df.csv').drop('Unnamed: 0', axis = 1)

In [12]:
def split(df):
  X = df.drop('Target', axis = 1)
  y = df.Target
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=42)
  return X_train, X_test, y_train, y_test

def metricas(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  return accuracy, precision

In [5]:
df = df_editado.filter(['Gender','scholarship', 'Target',  'Age at enrollment', 'Tasa_1st', 'Tasa_2nd'], axis = 1)
df['Course'] = dataset_original.Course
df = df[df.Target != 2].copy()

df_one = pd.get_dummies(df, columns = ['Course'])

scaler = StandardScaler()
colum_s= ['Age at enrollment', 'Tasa_1st', 'Tasa_2nd']
df_one = df_one.copy()
df_one[colum_s] = scaler.fit_transform(df_one[colum_s])

df_one.head()

,Gender,scholarship,Target,Age at enrollment,Tasa_1st,Tasa_2nd,Course_1,Course_2,Course_3,Course_4,...,Course_8,Course_9,Course_10,Course_11,Course_12,Course_13,Course_14,Course_15,Course_16,Course_17
0,1,0,0,-0.442212,-1.851221,-1.655320,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,-0.569976,0.793510,0.842316,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,-0.569976,-1.851221,-1.655320,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,-0.442212,0.793510,0.426044,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,1,2.751894,0.352721,0.842316,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
pca = PCA(n_components = 2)
pca.fit(df_one)
data_transf = pca.transform(df_one)

In [8]:
df_final = pd.DataFrame({'PCA1': data_transf[:, 0],
                         'PCA2': data_transf[:, 1],
                         'Target': df_one.Target
                         })
df_final.head()

,PCA1,PCA2,Target
0,2.415147,-1.209368,0
1,-1.207168,-0.125062,1
2,2.325101,-1.281370,0
3,-1.026252,-0.149233,1
4,-0.048068,2.940260,1


# Random Forrest

In [9]:
X_train, X_test, y_train, y_test = split(df_final)

In [10]:
modelo = RandomForestClassifier().fit(X_train, y_train)

In [11]:
y_pred = modelo.predict(X_test)

In [13]:
metricas(y_test, y_pred)

(0.9377245508982036, 0.9450757575757576)

# El resultado final es un modelo con un accuracy de 0.937 y una precision del 0.945

In [14]:
df_one.to_csv('Df_final.csv')